<a href="https://colab.research.google.com/github/q759729997/nlp_classification/blob/master/TNEWS_fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fastText is a library for efficient learning of word representations and sentence classification.

安装软件包

In [13]:
! pip install fasttext

     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2388660 sha256=e1b1b85b8c6f0adf908c763effda3cdf8c8e6da37d798557487859a887752192
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [19]:
! pip install jieba

In [0]:
- fasttext训练
- 参考：https://github.com/facebookresearch/fastText/tree/master/python
- 参考：https://pypi.org/project/fasttext/
- label prefix ['__label__']，标签与文本使用\t分割

读取数据

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import codecs
import random

In [0]:
def read_file_texts(file_name):
    "读取文件内容"""
    with codecs.open(file_name, mode='r', encoding='utf8') as fread:
        texts = list()
        for line in fread:
            line = line.strip()
            if len(line) > 0:
                texts.append(line)
        return texts

In [23]:
file_path = '/content/drive/My Drive/data/chineseGLUE/'
train_texts = read_file_texts(file_path + 'toutiao_category_train_v1.txt')
dev_texts = read_file_texts(file_path + 'toutiao_category_dev_v1.txt')
test_texts = read_file_texts(file_path + 'toutiao_category_test_v1.txt')
print('train_texts len:{}'.format(len(train_texts)))
print('train_texts example:{}'.format(train_texts[1]))
print('dev_texts len:{}'.format(len(dev_texts)))
print('dev_texts example:{}'.format(dev_texts[1]))
print('test_texts len:{}'.format(len(test_texts)))
print('test_texts example:{}'.format(test_texts[1]))

train_texts len:267881
train_texts example:news_entertainment	他是最帅的古装男神，10国语言六门武术，演技在线却常演配角！三生三世十里桃花,张智尧,杨门女将之女儿当自强,陆小凤,印象深刻,陆小凤传奇,杨宗保,花满楼,古剑奇谭
dev_texts len:57403
dev_texts example:news_sports	如果詹姆斯最巅峰的时候出现了，杜兰特是否能够顶得住压力呢？
test_texts len:57404
test_texts example:news_edu	2018年2月退休教师，教龄38年，退休金是多少？


In [0]:
def convert_to_ft_train_data(texts):
    """转换为fasttext训练所需数据格式"""
    ft_texts = list()
    for text in texts:
        label, feature = text.split('\t')
        # 分词，使用单个字符
        tokens = list(feature.replace('\t', ''))
        # 组装数据
        ft_texts.append('__label__{}\t{}'.format(label, ' '.join(tokens)))
    return ft_texts

In [0]:
import jieba

In [0]:
def convert_to_ft_train_data(texts):
    """转换为fasttext训练所需数据格式"""
    ft_texts = list()
    for text in texts:
        label, feature = text.split('\t')
        # jieba分词
        tokens = jieba.cut(feature.replace('\t', ''))
        # 组装数据
        ft_texts.append('__label__{}\t{}'.format(label, ' '.join(tokens)))
    return ft_texts

In [25]:
train_data = convert_to_ft_train_data(train_texts)
train_data.extend(convert_to_ft_train_data(dev_texts))
test_data = convert_to_ft_train_data(test_texts)
print('train_data len:{}'.format(len(train_data)))
print('train_data example:{}'.format(train_data[1]))
print('test_data len:{}'.format(len(test_data)))
print('test_data example:{}'.format(test_data[1]))

Building prefix dict from the default dictionary ...
2019-11-01 15:01:23,426 : DEBUG : Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2019-11-01 15:01:24,433 : DEBUG : Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.090 seconds.
2019-11-01 15:01:24,522 : DEBUG : Loading model cost 1.090 seconds.
Prefix dict has been built succesfully.
2019-11-01 15:01:24,524 : DEBUG : Prefix dict has been built succesfully.


train_data len:325284
train_data example:__label__news_entertainment	他 是 最帅 的 古装 男神 ， 10 国 语言 六门 武术 ， 演技 在线 却常演 配角 ！ 三生 三世 十里 桃花 , 张智尧 , 杨门 女将 之 女儿 当自强 , 陆小凤 , 印象 深刻 , 陆小凤 传奇 , 杨宗保 , 花满楼 , 古剑 奇谭
test_data len:57404
test_data example:__label__news_edu	2018 年 2 月 退休 教师 ， 教龄 38 年 ， 退休金 是 多少 ？


In [0]:
def output_ft_train_data(file_name, texts):
    """输出fasttext训练文件"""
    with codecs.open(file_name, mode='w', encoding='utf8') as fw:
        for text in texts:
            fw.write('{}\n'.format(text))

In [0]:
file_path = '/content/sample_data/'
train_data_file = file_path + 'toutiao_train_ft_chars.txt'
test_data_file = file_path + 'toutiao_test_ft_chars.txt'
train_texts = output_ft_train_data(train_data_file, train_data)
dev_texts = output_ft_train_data(test_data_file, test_data)

In [28]:
print(train_data_file)
print(test_data_file)

/content/sample_data/toutiao_train_ft_chars.txt
/content/sample_data/toutiao_test_ft_chars.txt


In [29]:
! head -2 /content/sample_data/toutiao_train_ft_chars.txt

__label__news_finance	股票 中 的 突破 形态 股票
__label__news_entertainment	他 是 最帅 的 古装 男神 ， 10 国 语言 六门 武术 ， 演技 在线 却常演 配角 ！ 三生 三世 十里 桃花 , 张智尧 , 杨门 女将 之 女儿 当自强 , 陆小凤 , 印象 深刻 , 陆小凤 传奇 , 杨宗保 , 花满楼 , 古剑 奇谭


In [30]:
! head -2 /content/sample_data/toutiao_test_ft_chars.txt

__label__news_military	歼 20 座 舱盖 上 的 两条 “ 花纹 ” 是 什么 ？
__label__news_edu	2018 年 2 月 退休 教师 ， 教龄 38 年 ， 退休金 是 多少 ？


模型训练

In [4]:
import fasttext
# 日志初始化
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.info('logging test')

2019-11-01 15:14:13,387 : INFO : logging test


In [0]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [10]:
# 默认参数
logging.info('start training')
model = fasttext.train_supervised(input=train_data_file)
logging.info('end training')
print_results(*model.test(test_data_file))

2019-11-01 15:15:01,422 : INFO : start training
2019-11-01 15:15:35,504 : INFO : end training


N	57404
P@1	0.901
R@1	0.901


In [32]:
# wordNgrams=2
logging.info('start training')
model = fasttext.train_supervised(input=train_data_file, wordNgrams=2)
logging.info('end training')
print_results(*model.test(test_data_file))

2019-11-01 15:03:52,944 : INFO : start training
2019-11-01 15:04:53,291 : INFO : end training


N	57404
P@1	0.907
R@1	0.907


In [33]:
# wordNgrams=3
logging.info('start training')
model = fasttext.train_supervised(input=train_data_file, wordNgrams=3)
logging.info('end training')
print_results(*model.test(test_data_file))

2019-11-01 15:04:58,672 : INFO : start training
2019-11-01 15:06:23,094 : INFO : end training


N	57404
P@1	0.905
R@1	0.905


## 单字特征

- 默认参数
~~~
N	57404
P@1	0.859
R@1	0.859
~~~
- wordNgrams=2
~~~
N	57404
P@1	0.903
R@1	0.903
~~~

- wordNgrams=2
~~~
N	57404
P@1	0.906
R@1	0.906
~~~

## jieba分词特征

- 默认参数
~~~
N	57404
P@1	0.901
R@1	0.901
~~~
- wordNgrams=2
~~~
N	57404
P@1	0.907
R@1	0.907
~~~

- wordNgrams=2
~~~
N	57404
P@1	0.905
R@1	0.905
~~~

模型测试与保存

In [34]:
# 预测测试
model.predict(' '.join(list("一起看周杰伦的演唱会")))

(('__label__news_entertainment',), array([0.97608125]))

In [0]:
# 模型保存
model.save_model("/content/sample_data/toutiao_ft.ftbin")

In [11]:
# 模型压缩
# with the previously trained `model` object, call :
logging.info('start training')
model.quantize(train_data_file, retrain=True)
logging.info('end training')
# then display results and save the new model :
print_results(*model.test(test_data_file))

2019-11-01 15:15:43,975 : INFO : start training
2019-11-01 15:17:13,339 : INFO : end training


N	57404
P@1	0.900
R@1	0.900


In [0]:
model.save_model("/content/sample_data/model_filename.ftz")

In [13]:
! ls -lh /content/sample_data/

total 977M
-rwxr-xr-x 1 root root 1.7K Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root 295K Oct 25 16:58 california_housing_test.csv
-rw-r--r-- 1 root root 1.7M Oct 25 16:58 california_housing_train.csv
-rw-r--r-- 1 root root  18M Oct 25 16:58 mnist_test.csv
-rw-r--r-- 1 root root  35M Oct 25 16:58 mnist_train_small.csv
-rw-r--r-- 1 root root  15M Nov  1 15:17 model_filename.ftz
-rwxr-xr-x 1 root root  930 Jan  1  2000 README.md
-rw-r--r-- 1 root root 855M Nov  1 15:08 toutiao_ft.ftbin
-rw-r--r-- 1 root root 8.0M Nov  1 15:03 toutiao_test_ft_chars.txt
-rw-r--r-- 1 root root  46M Nov  1 15:03 toutiao_train_ft_chars.txt


可以看出：模型由原来的855M，压缩至15M，而评分基本没变